## Data 전처리

### 기업 통일

2016년을 기준으로 2017,2018,2019의 기업을 통일 시킬 것이다.

2016년 이후 진규 창립기업은 모델 학습, 테스트에 사용하지 않을 것이며, 파산기업은 파산한 시점부터 모든 값들을 0으로 처리한다. 뿅

In [1]:
import pandas as pd
import numpy as np

In [2]:
standard_data = pd.read_excel('C:/Users/jlee0/Desktop/KYU/hanim ict/1/2017.01.xlsx', encoding= "utf-8")

In [3]:
standard_data.head()

,코드\n번호,회사명,업종\n(대),업종\n(소),코스피코스닥,주가\n(원),시가총액\n(억),1일\n등락률\n(%),5일\n등락률\n(%),20일\n등락률\n(%),...,16년 3Q.2,16년 4Q(E).2,17년 1Q(E).2,16년4Q \n매출액,16년4Q \n영업이익,16년4Q \n순이익,16년4Q \n매출액.1,16년4Q \n영업이익.1,16년4Q \n순이익.1,16년4Q \n순이익.2
0,A005930,삼성전자,반도체 관련장비 및 부품,종합 반도체,코스피,1833000,2578652,-2.135611,-1.504567,2.230898,...,44087.90,64474.315237,66685.180236,530000.0,92000.0,NaN,521120.451352,83715.404235,64474.315237,64474.315237
1,A000660,SK하이닉스,반도체 관련장비 및 부품,종합 반도체,코스피,49300,358905,-1.988072,-0.504541,6.250000,...,5967.74,10690.789385,12736.206023,NaN,NaN,NaN,51332.259150,14568.549707,10690.789385,10690.789385
2,A005380,현대차,자동차 및 관련부품,완성차,코스피,150500,331516,1.006711,0.000000,7.885305,...,10614.40,13513.247779,14784.204410,NaN,NaN,NaN,247224.041550,13353.916707,13513.247779,13513.247779
3,A015760,한국전력,유틸리티,전력,코스피,44400,285032,-0.448430,2.896871,-3.687636,...,29133.61,13654.365561,17147.707040,NaN,NaN,NaN,147316.591167,24639.338148,13654.365561,13654.365561
4,A035420,NAVER,IT서비스,인터넷서비스,코스피,794000,261724,0.506329,1.925546,-1.488834,...,1945.48,2117.854524,2221.450690,NaN,NaN,NaN,10916.344137,3030.094202,2117.854524,2117.854524


In [4]:
standard_data['Code'] = standard_data['코드\n번호']

In [5]:
standard_data.head()

,코드\n번호,회사명,업종\n(대),업종\n(소),코스피코스닥,주가\n(원),시가총액\n(억),1일\n등락률\n(%),5일\n등락률\n(%),20일\n등락률\n(%),...,16년 4Q(E).2,17년 1Q(E).2,16년4Q \n매출액,16년4Q \n영업이익,16년4Q \n순이익,16년4Q \n매출액.1,16년4Q \n영업이익.1,16년4Q \n순이익.1,16년4Q \n순이익.2,Code
0,A005930,삼성전자,반도체 관련장비 및 부품,종합 반도체,코스피,1833000,2578652,-2.135611,-1.504567,2.230898,...,64474.315237,66685.180236,530000.0,92000.0,NaN,521120.451352,83715.404235,64474.315237,64474.315237,A005930
1,A000660,SK하이닉스,반도체 관련장비 및 부품,종합 반도체,코스피,49300,358905,-1.988072,-0.504541,6.250000,...,10690.789385,12736.206023,NaN,NaN,NaN,51332.259150,14568.549707,10690.789385,10690.789385,A000660
2,A005380,현대차,자동차 및 관련부품,완성차,코스피,150500,331516,1.006711,0.000000,7.885305,...,13513.247779,14784.204410,NaN,NaN,NaN,247224.041550,13353.916707,13513.247779,13513.247779,A005380
3,A015760,한국전력,유틸리티,전력,코스피,44400,285032,-0.448430,2.896871,-3.687636,...,13654.365561,17147.707040,NaN,NaN,NaN,147316.591167,24639.338148,13654.365561,13654.365561,A015760
4,A035420,NAVER,IT서비스,인터넷서비스,코스피,794000,261724,0.506329,1.925546,-1.488834,...,2117.854524,2221.450690,NaN,NaN,NaN,10916.344137,3030.094202,2117.854524,2117.854524,A035420


In [7]:
test_data = pd.read_excel('C:/Users/jlee0/Desktop/KYU/hanim ict/1/sampo_monthly/sampo_monthly/2018/2018.01.01.xlsx', encoding= "utf-8")

In [8]:
test_data['Code'] = test_data['코드\n번호']

In [9]:
test_data.head()

,코드\n번호,회사명,업종\n(대),업종\n(소),코스피코스닥,주가\n(원),시가총액\n(억),상장주식수\n(만주),자사주\n(만주),자사주\n비중\n(%),...,17년 4Q(E).2,18년 1Q(E).2,17년4Q \n매출액,17년4Q \n영업이익,17년4Q \n순이익,17년4Q \n매출액.1,17년4Q \n영업이익.1,17년4Q \n순이익.1,17년4Q \n순이익.2,Code
0,A005930,삼성전자,반도체 관련장비 및 부품,종합 반도체,코스피,2466000,3183568.86,12909.849392,941.7123,7.294526,...,116317.0,123393.0,660000.0,151000.0,NaN,668200.0,156882.0,116317.0,116317.0,A005930
1,A000660,SK하이닉스,반도체 관련장비 및 부품,종합 반도체,코스피,73300,533625.73,72800.236016,2200.0570,3.022047,...,34525.0,33059.0,NaN,NaN,NaN,89812.0,43506.0,34525.0,34525.0,A000660
2,A005380,현대차,자동차 및 관련부품,완성차,코스피,162000,356847.90,22027.648148,1322.2514,6.002690,...,10743.0,11752.0,NaN,NaN,NaN,239459.0,11104.0,10743.0,10743.0,A005380
3,A005490,POSCO,철강 및 비철강,제철,코스피,385500,336105.25,8718.683528,718.7231,8.243482,...,7465.0,8421.0,NaN,NaN,NaN,155474.0,12626.0,7465.0,7465.0,A005490
4,A035420,NAVER,IT서비스,인터넷서비스,코스피,881000,290401.20,3296.267877,359.2203,10.897788,...,2202.0,2313.0,NaN,NaN,NaN,12493.0,3174.0,2202.0,2202.0,A035420


In [10]:
type(test_data['Code'])

pandas.core.series.Series

In [11]:
standard_bis = set(standard_data['Code'])

In [12]:
test_bis = set(test_data['Code'])

In [13]:
test_bis - standard_bis

{'A002800',
 'A003380',
 'A063760',
 'A079440',
 'A083500',
 'A087260',
 'A091990',
 'A111110',
 'A118990',
 'A140070',
 'A140670',
 'A145720',
 'A161580',
 'A166090',
 'A171090',
 'A174900',
 'A178320',
 'A179900',
 'A181340',
 'A183300',
 'A206650',
 'A217480',
 'A225190',
 'A227610',
 'A234100',
 'A238490',
 'A241820',
 'A243840',
 'A246720',
 'A250000',
 'A251270',
 'A251370',
 'A251630',
 'A255440',
 'A256630',
 'A256840',
 'A256940',
 'A257730',
 'A258610',
 'A258790',
 'A259630',
 'A261200',
 'A262830',
 'A263540',
 'A263600',
 'A263720',
 'A263750',
 'A263770',
 'A263800',
 'A263810',
 'A263860',
 'A263920',
 'A264290',
 'A264450',
 'A264850',
 'A264900',
 'A265480',
 'A265520',
 'A265560',
 'A265920',
 'A267250',
 'A267260',
 'A267270',
 'A267290',
 'A267320',
 'A267980',
 'A268280',
 'A270520',
 'A271560',
 'A271740',
 'A271980',
 'A272290',
 'A273060',
 'A277410',
 'A277480',
 'A280360',
 'A900310'}

In [14]:
drop_bis = test_bis - standard_bis  #신규 발행 회사 제거
for k in drop_bis:
    id = test_data[test_data['Code']==k].index
    test_data.drop(id,inplace = True)

In [15]:
test_data.head()

,코드\n번호,회사명,업종\n(대),업종\n(소),코스피코스닥,주가\n(원),시가총액\n(억),상장주식수\n(만주),자사주\n(만주),자사주\n비중\n(%),...,17년 4Q(E).2,18년 1Q(E).2,17년4Q \n매출액,17년4Q \n영업이익,17년4Q \n순이익,17년4Q \n매출액.1,17년4Q \n영업이익.1,17년4Q \n순이익.1,17년4Q \n순이익.2,Code
0,A005930,삼성전자,반도체 관련장비 및 부품,종합 반도체,코스피,2466000,3183568.86,12909.849392,941.7123,7.294526,...,116317.0,123393.0,660000.0,151000.0,NaN,668200.0,156882.0,116317.0,116317.0,A005930
1,A000660,SK하이닉스,반도체 관련장비 및 부품,종합 반도체,코스피,73300,533625.73,72800.236016,2200.0570,3.022047,...,34525.0,33059.0,NaN,NaN,NaN,89812.0,43506.0,34525.0,34525.0,A000660
2,A005380,현대차,자동차 및 관련부품,완성차,코스피,162000,356847.90,22027.648148,1322.2514,6.002690,...,10743.0,11752.0,NaN,NaN,NaN,239459.0,11104.0,10743.0,10743.0,A005380
3,A005490,POSCO,철강 및 비철강,제철,코스피,385500,336105.25,8718.683528,718.7231,8.243482,...,7465.0,8421.0,NaN,NaN,NaN,155474.0,12626.0,7465.0,7465.0,A005490
4,A035420,NAVER,IT서비스,인터넷서비스,코스피,881000,290401.20,3296.267877,359.2203,10.897788,...,2202.0,2313.0,NaN,NaN,NaN,12493.0,3174.0,2202.0,2202.0,A035420


In [16]:
#제대로 기업이 빠졌는지 확인

if 'A002800' in set(test_data['Code']):
    print('Fail..')
else:
    print('Success')

Success


In [17]:
test_data

,코드\n번호,회사명,업종\n(대),업종\n(소),코스피코스닥,주가\n(원),시가총액\n(억),상장주식수\n(만주),자사주\n(만주),자사주\n비중\n(%),...,17년 4Q(E).2,18년 1Q(E).2,17년4Q \n매출액,17년4Q \n영업이익,17년4Q \n순이익,17년4Q \n매출액.1,17년4Q \n영업이익.1,17년4Q \n순이익.1,17년4Q \n순이익.2,Code
0,A005930,삼성전자,반도체 관련장비 및 부품,종합 반도체,코스피,2466000,3183568.86,12909.849392,941.7123,7.294526,...,116317.0,123393.0,660000.0,151000.0,NaN,668200.0,156882.0,116317.0,116317.0,A005930
1,A000660,SK하이닉스,반도체 관련장비 및 부품,종합 반도체,코스피,73300,533625.73,72800.236016,2200.0570,3.022047,...,34525.0,33059.0,NaN,NaN,NaN,89812.0,43506.0,34525.0,34525.0,A000660
2,A005380,현대차,자동차 및 관련부품,완성차,코스피,162000,356847.90,22027.648148,1322.2514,6.002690,...,10743.0,11752.0,NaN,NaN,NaN,239459.0,11104.0,10743.0,10743.0,A005380
3,A005490,POSCO,철강 및 비철강,제철,코스피,385500,336105.25,8718.683528,718.7231,8.243482,...,7465.0,8421.0,NaN,NaN,NaN,155474.0,12626.0,7465.0,7465.0,A005490
4,A035420,NAVER,IT서비스,인터넷서비스,코스피,881000,290401.20,3296.267877,359.2203,10.897788,...,2202.0,2313.0,NaN,NaN,NaN,12493.0,3174.0,2202.0,2202.0,A035420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1989,A226440,동부스팩3호,금융,스팩,코스닥,2055,91.45,445.012165,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A226440
1992,A218150,미래생명자원,금융,스팩,코스닥,3730,637.88,1710.134048,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A218150
1994,A225440,이베스트스팩3호,금융,스팩,코스닥,2005,82.21,410.024938,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A225440
1998,A232330,SK3호스팩,금융,스팩,코스닥,2015,72.54,360.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A232330


In [18]:
def select_bis(standard_data,test_data):  #standard_data : 2017.01월 데이터, test_bis : 원하는 일시의 데이터
    standard_bis = set(standard_data['Code'])
    test_bis = set(test_data['Code'])
    drop_bis = test_bis - standard_bis  #신규 발행 회사 제거
    for k in drop_bis:
        id = test_data[test_data['Code']==k].index
        test_data.drop(id,inplace = True)
        
    return test_data

### 피쳐 통일

In [19]:
reverse_st_data = standard_data.T
reverse_te_data = test_data.T
standard_data.T.head()
test_data.T.head()

,0,1,2,3,4,5,6,7,8,9,...,1982,1984,1985,1987,1988,1989,1992,1994,1998,1999
코드\n번호,A005930,A000660,A005380,A005490,A035420,A051910,A028260,A032830,A207940,A015760,...,A217810,A239340,A217500,A254120,A225430,A226440,A218150,A225440,A232330,A215100
회사명,삼성전자,SK하이닉스,현대차,POSCO,NAVER,LG화학,삼성물산,삼성생명,삼성바이오로직스,한국전력,...,엔에이치스팩7호,미래에셋제5호스팩,하이제3호스팩,IBKS제5호스팩,IBKS제3호스팩,동부스팩3호,미래생명자원,이베스트스팩3호,SK3호스팩,로보로보
업종\n(대),반도체 관련장비 및 부품,반도체 관련장비 및 부품,자동차 및 관련부품,철강 및 비철강,IT서비스,화학,건설 및 건축자재,금융,제약 및 바이오,유틸리티,...,금융,금융,금융,금융,금융,금융,금융,금융,금융,금융
업종\n(소),종합 반도체,종합 반도체,완성차,제철,인터넷서비스,종합 화학,대형건설사,생명보험,바이오,전력,...,스팩,스팩,스팩,스팩,스팩,스팩,스팩,스팩,스팩,스팩
코스피코스닥,코스피,코스피,코스피,코스피,코스피,코스피,코스피,코스피,코스피,코스피,...,코스닥,코스닥,코스닥,코스닥,코스닥,코스닥,코스닥,코스닥,코스닥,코스닥


In [20]:
reverse_st_data.reset_index(inplace = True)
reverse_te_data.reset_index(inplace = True)

In [21]:
standard_features = set(reverse_st_data['index'])
test_features = set(reverse_te_data['index'])
drop_features = test_features - standard_features  #신규 발행 피쳐 제거
for k in drop_features:
    id = reverse_te_data[reverse_te_data['index']==k].index
    reverse_te_data.drop(id,inplace = True)

In [22]:
drop_features

{'13년->16년\n3년간 YOY',
 '13년->16년\n3년간 YOY.1',
 '13년->16년\n3년간 YOY.2',
 '16년\nYOY',
 '16년\nYOY.1',
 '16년\nYOY.2',
 '16년\n매출액',
 '16년\n순이익',
 '16년\n영업이익',
 '16년 4Q',
 '16년 4Q.1',
 '16년 4Q.2',
 '16년4Q QOQ',
 '16년4Q QOQ.1',
 '16년4Q QOQ.2',
 '16년4Q YOY',
 '16년4Q YOY.1',
 '16년4Q YOY.2',
 '17/3Q\n매출액\n컨센대비',
 '17/3Q\n순이익\n컨센대비',
 '17/3Q\n영업이익\n컨센대비',
 '17/4Q\n매출액\n컨센대비',
 '17/4Q\n순이익\n컨센대비',
 '17/4Q\n영업이익\n컨센대비',
 '17년\nOPM\n(%)',
 '17년\nPER',
 '17년 1Q',
 '17년 1Q.1',
 '17년 1Q.2',
 '17년 2Q',
 '17년 2Q.1',
 '17년 2Q.2',
 '17년 3Q',
 '17년 3Q.1',
 '17년 3Q.2',
 '17년 4Q(E)',
 '17년 4Q(E).1',
 '17년 4Q(E).2',
 '17년1Q QOQ',
 '17년1Q QOQ.1',
 '17년1Q QOQ.2',
 '17년1Q YOY',
 '17년1Q YOY.1',
 '17년1Q YOY.2',
 '17년2Q QOQ',
 '17년2Q QOQ.1',
 '17년2Q QOQ.2',
 '17년2Q YOY',
 '17년2Q YOY.1',
 '17년2Q YOY.2',
 '17년3Q\n호실적\n100기업',
 '17년3Q QOQ',
 '17년3Q QOQ.1',
 '17년3Q QOQ.2',
 '17년3Q YOY',
 '17년3Q YOY.1',
 '17년3Q YOY.2',
 '17년4Q\n호실적예상\n100기업',
 '17년4Q \n매출액',
 '17년4Q \n매출액.1',
 '17년4Q \n순이익',
 '17년4Q \n순이익.1',
 '17년4Q \n순이

In [23]:
#제대로 피쳐가 빠졌는지 확인

if '총자산\n(억)' in set(reverse_te_data['index']):
    print('Fail..')
else:
    print('Success')

Success


In [24]:
#다시 원래대로 되돌리기

test_data = reverse_te_data.T

In [25]:
test_data

,0,1,2,3,4,5,6,11,12,23,...,202,209,210,211,212,219,220,221,222,236
index,코드\n번호,회사명,업종\n(대),업종\n(소),코스피코스닥,주가\n(원),시가총액\n(억),1일\n등락률\n(%),5일\n등락률\n(%),과거\nROE\n(%),...,16년 3Q,15년 4Q.1,16년 1Q.1,16년 2Q.1,16년 3Q.1,15년 4Q.2,16년 1Q.2,16년 2Q.2,16년 3Q.2,Code
0,A005930,삼성전자,반도체 관련장비 및 부품,종합 반도체,코스피,2466000,3.18357e+06,-1.16,2.32365,17.8,...,478156,61427.7,66758.1,81439.5,52000.9,32424.7,52635.1,58261.8,44087.9,A005930
1,A000660,SK하이닉스,반도체 관련장비 및 부품,종합 반도체,코스피,73300,533626,-2.53,-1.47849,29.3,...,42436.4,9888.75,5617.98,4528.53,7259.56,8702.5,4441.94,2861.41,5967.74,A000660
2,A005380,현대차,자동차 및 관련부품,완성차,코스피,162000,356848,4.52,5.19481,5.8,...,220837,15150.5,13423.9,17618.4,10681,16314.2,16870.3,16579.6,10614.4,A005380
3,A005490,POSCO,철강 및 비철강,제철,코스피,385500,336105,0.13,-0.258732,5.5,...,127476,3405.46,6597.66,6785.14,10343,1855.1,3596.71,2916.54,5456.89,A005490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1989,A226440,동부스팩3호,금융,스팩,코스닥,2055,91.45,0,0,0.3,...,0,-0.22,-0.12,-0.13,-0.08,-0.3,0.04,-0.12,0.14,A226440
1992,A218150,미래생명자원,금융,스팩,코스닥,3730,637.88,-5.21,3.03867,-0.1,...,0,-0.1,-0.01,-0.04,-0.03,0.12,0.11,0.18,0.25,A218150
1994,A225440,이베스트스팩3호,금융,스팩,코스닥,2005,82.21,-0.5,0,0.8,...,0,0,0,-0.22,-0.09,0,0,0.06,0.16,A225440
1998,A232330,SK3호스팩,금융,스팩,코스닥,2015,72.54,0,0.498753,0,...,0,0,-0.27,-0.13,-0.1,0,-0.22,-0.13,-0.11,A232330


In [26]:
def select_feature(standard_data,test_data):  #standard_data : 2017.01월 데이터, test_bis : 원하는 일시의 데이터
    reverse_st_data = standard_data.T
    reverse_te_data = test_data.T
    standard_data.T.head()
    test_data.T.head()
    standard_features = set(reverse_st_data['index'])
    test_features = set(reverse_te_data['index'])
    drop_features = test_features - standard_features  #신규 발행 피쳐 제거
    for k in drop_features:
        id = reverse_te_data[reverse_te_data['index']==k].index
        reverse_te_data.drop(id,inplace = True)
    test_data = reverse_test_data.T
    
    return test_data

In [30]:
import pandas as pd
import numpy as np

standard_data = pd.read_excel('C:/Users/jlee0/Desktop/KYU/hanim ict/1/2017.01.xlsx', encoding= "utf-8")
standard_data['Code'] = standard_data['코드\n번호']
test_data = pd.read_excel('C:/Users/jlee0/Desktop/KYU/hanim ict/1/sampo_monthly/sampo_monthly/2018/2018.01.01.xlsx', encoding= "utf-8")
test_data['Code'] = test_data['코드\n번호']

def select_bis(standard_data,test_data):  #standard_data : 2017.01월 데이터, test_bis : 원하는 일시의 데이터
    standard_bis = set(standard_data['Code'])
    test_bis = set(test_data['Code'])
    drop_bis = test_bis - standard_bis  #신규 발행 회사 제거
    for k in drop_bis:
        id = test_data[test_data['Code']==k].index
        test_data.drop(id,inplace = True)
        
    return test_data

def select_feature(standard_data,test_data):  #standard_data : 2017.01월 데이터, test_bis : 원하는 일시의 데이터
    reverse_st_data = standard_data.T
    reverse_te_data = test_data.T
    standard_data.T.head()
    test_data.T.head()
    standard_features = set(reverse_st_data['index'])
    test_features = set(reverse_te_data['index'])
    drop_features = test_features - standard_features  #신규 발행 피쳐 제거
    for k in drop_features:
        id = reverse_te_data[reverse_te_data['index']==k].index
        reverse_te_data.drop(id,inplace = True)
    test_data = reverse_test_data.T
    
    return test_data

import openpyxl

test_data = select_bis(standard_data,test_data)
test_data = select_bis(standard_data,test_data)

test_data.to_excel('result.2018_01.xlsx')

test_data = pd.read_excel('C:/Users/jlee0/Desktop/KYU/hanim ict/1/sampo_monthly/sampo_monthly/2018/2018.02.01.xlsx', encoding= "utf-8")
test_data['Code'] = test_data['코드\n번호']

test_data = select_bis(standard_data,test_data)
test_data = select_bis(standard_data,test_data)

test_data.to_excel('result.2018_02.xlsx')

KeyboardInterrupt: 

In [31]:
test_data = select_bis(standard_data,test_data)
test_data = select_bis(standard_data,test_data)

test_data.to_excel('result.2018_01.xlsx')

test_data = pd.read_excel('C:/Users/jlee0/Desktop/KYU/hanim ict/1/sampo_monthly/sampo_monthly/2018/2018.02.01.xlsx', encoding= "utf-8")
test_data['Code'] = test_data['코드\n번호']

test_data = select_bis(standard_data,test_data)
test_data = select_bis(standard_data,test_data)

test_data.to_excel('result.2018_02.xlsx')

In [32]:
test_data = pd.read_excel('C:/Users/jlee0/Desktop/KYU/hanim ict/1/sampo_monthly/sampo_monthly/2018/2018.03.01.xlsx', encoding= "utf-8")
test_data['Code'] = test_data['코드\n번호']

test_data = select_bis(standard_data,test_data)
test_data = select_bis(standard_data,test_data)

test_data.to_excel('result.2018_03.xlsx')

In [33]:
test_data = pd.read_excel('C:/Users/jlee0/Desktop/KYU/hanim ict/1/sampo_monthly/sampo_monthly/2018/2018.04.01.xlsx', encoding= "utf-8")
test_data['Code'] = test_data['코드\n번호']

test_data = select_bis(standard_data,test_data)
test_data = select_bis(standard_data,test_data)

test_data.to_excel('result.2018_04.xlsx')

In [34]:
test_data = pd.read_excel('C:/Users/jlee0/Desktop/KYU/hanim ict/1/sampo_monthly/sampo_monthly/2018/2018.05.01.xlsx', encoding= "utf-8")
test_data['Code'] = test_data['코드\n번호']

test_data = select_bis(standard_data,test_data)
test_data = select_bis(standard_data,test_data)

test_data.to_excel('result.2018_05.xlsx')

In [35]:
test_data = pd.read_excel('C:/Users/jlee0/Desktop/KYU/hanim ict/1/sampo_monthly/sampo_monthly/2018/2018.06.01.xlsx', encoding= "utf-8")
test_data['Code'] = test_data['코드\n번호']

test_data = select_bis(standard_data,test_data)
test_data = select_bis(standard_data,test_data)

test_data.to_excel('result.2018_06.xlsx')

In [37]:
test_data = pd.read_excel('C:/Users/jlee0/Desktop/KYU/hanim ict/1/sampo_monthly/sampo_monthly/2018/2018.07.01.xlsx', encoding= "utf-8")
test_data['Code'] = test_data['코드\n번호']

test_data = select_bis(standard_data,test_data)
test_data = select_bis(standard_data,test_data)

test_data.to_excel('result.2018_07.xlsx')

In [38]:
test_data = pd.read_excel('C:/Users/jlee0/Desktop/KYU/hanim ict/1/sampo_monthly/sampo_monthly/2018/2018.08.01.xlsx', encoding= "utf-8")
test_data['Code'] = test_data['코드\n번호']

test_data = select_bis(standard_data,test_data)
test_data = select_bis(standard_data,test_data)

test_data.to_excel('result.2018_08.xlsx')

In [39]:
test_data = pd.read_excel('C:/Users/jlee0/Desktop/KYU/hanim ict/1/sampo_monthly/sampo_monthly/2018/2018.09.01.xlsx', encoding= "utf-8")
test_data['Code'] = test_data['코드\n번호']

test_data = select_bis(standard_data,test_data)
test_data = select_bis(standard_data,test_data)

test_data.to_excel('result.2018_09.xlsx')

In [40]:
test_data = pd.read_excel('C:/Users/jlee0/Desktop/KYU/hanim ict/1/sampo_monthly/sampo_monthly/2018/2018.10.01.xlsx', encoding= "utf-8")
test_data['Code'] = test_data['코드\n번호']

test_data = select_bis(standard_data,test_data)
test_data = select_bis(standard_data,test_data)

test_data.to_excel('result.2018_10.xlsx')

In [41]:
test_data = pd.read_excel('C:/Users/jlee0/Desktop/KYU/hanim ict/1/sampo_monthly/sampo_monthly/2018/2018.11.01.xlsx', encoding= "utf-8")
test_data['Code'] = test_data['코드\n번호']

test_data = select_bis(standard_data,test_data)
test_data = select_bis(standard_data,test_data)

test_data.to_excel('result.2018_11.xlsx')

In [42]:
test_data = pd.read_excel('C:/Users/jlee0/Desktop/KYU/hanim ict/1/sampo_monthly/sampo_monthly/2018/2018.12.01.xlsx', encoding= "utf-8")
test_data['Code'] = test_data['코드\n번호']

test_data = select_bis(standard_data,test_data)
test_data = select_bis(standard_data,test_data)

test_data.to_excel('result.2018_12.xlsx')

In [43]:
if 'A002800' in set(test_data['Code']):
    print('Fail..')
else:
    print('Success')

Success
